1 задание

Имеется файл с временной статистикой работы асессоров над однотипным заданием.

Формат файла: **login tid Microtasks assigned_ts tclosed_ts.**

Пояснение к формату: 

**login** — логин асессора; 

**tid** — id оцениваемого задания (task id); 

**Microtasks** – количество микрозаданий в одном задании; 

**assigned_ts** — время резервирования системой задания для асессора; 

**closed_ts** — точное время завершения работы над заданием; 

разделитель — табуляция **\t**.


Задание может состоять из одного или несколько микрозаданий. 

Время резервирования задания (assigned_ts) указывает на тот момент, когда система назначила определенного асессора исполнителем этого задания. Этот момент может совпадать с временем начала работы асессора над заданием, а может и не совпадать (асессор может отойти выпить чаю, а потом приступить к заданию, асессор может выполнять предыдущее задание, в то время как за ним зарезервированы новые).

Предположим, что асессор за 30 секунд своего рабочего времени получает N рублей.

Какую оплату вы считаете справедливой для выполнения асессором одного микрозадания из этого файла? 

Опишите подробно все этапы вашего решения.

Приложите, пожалуйста, исходный код решения задачи или приведите примеры частей кода в описании решения.


Импорт необходимых библиотек

In [1]:
import pandas as pd
import csv
import numpy as np
from datetime import datetime

Получение доступа к файлу с данными

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
PATH_TO_FILE = "drive/MyDrive/Colab Data/yandex/first.txt"
data = pd.read_csv(PATH_TO_FILE, sep="\t")

In [4]:
PAY_FOR_30_SECONDS = 1
start_time = datetime(1970, 1, 1)
template = '%Y-%m-%d %H:%M:%S'

Как будет происходить вычисление стоимости выполнения микрозадания?

Для этого я посмотрю,
сколько времени в среднем тратится на выполнения одного микрозадания у каждого пользователя.
После этого я возьму медиану этих значений у всех пользователей, то есть время T, которое затрачивает
пользователь на выполнение одного микрозадания.

Из этого же можно узнать стоимость поделив это время на 30 секунд.
answer = (T / 30) * N

In [5]:
def get_payment(lead_time):
    return (lead_time / 30) * PAY_FOR_30_SECONDS

Самым очевидным и простым вариантом реализации является вариант без учета пересечений
временных отрезков выполнения различных заданий.

Преобразовал столбцы с датами для удобного использования

In [6]:
data.assigned_ts = data.assigned_ts.apply(lambda ts: datetime.strptime(ts, '%Y-%m-%d %H:%M:%S'))
data.closed_ts = data.closed_ts.apply(lambda ts: datetime.strptime(ts, '%Y-%m-%d %H:%M:%S'))

Посчитал сколько времени уходило на одно микрозадание

In [7]:
data['new_time'] = data.closed_ts - data.assigned_ts
data['new_time_in_seconds'] = data.new_time.apply(lambda ts: ts.total_seconds())
data['seconds_for_microtask'] = data.new_time_in_seconds / data.Microtasks

Если взять медиану от столбца seconds_for_microtask, то в результате мы увидим, что 75 секунд тратится
на выполнение одного микрозадания.

In [8]:
median_time_first = data.seconds_for_microtask.median()
median_time_first

75.0

То есть в данном случае одно микрозадание будет стоить payment_first

In [12]:
payment_first = get_payment(median_time_first)
print("Оплата за одно микрозадание(первый вариант) =", payment_first, 'N')

Оплата за одно микрозадание(первый вариант) = 2.5 N


Но в предыдущем варианте решения не учитываются особенности каждого из ассесоров. Некоторые из пользователей могут выполнять большое количество заданий в своем темпе, будь то быстро или медленно. Поэтому предлагается сгруппировать задания по пользователям и взять время выполнения микрозаданий сначала медианное у каждого пользователя, а после уже у всех взять медиану полученных значений

In [15]:
median_time_by_login = data.groupby('login').aggregate('seconds_for_microtask').median()

login
login0      239.000000
login1      100.937500
login10      92.000000
login100    101.583333
login101     65.000000
               ...    
login95      47.000000
login96      38.222222
login97     784.800000
login98     709.333333
login99      98.388889
Name: seconds_for_microtask, Length: 767, dtype: float64

In [22]:
median_time_second = median_time_by_login.median() 
median_time_second

110.75

In [19]:
payment_second = get_payment(median_time_second)
print("Оплата за одно микрозадание(второй вариант) =", payment_second, 'N')

Оплата за одно микрозадание(второй вариант) = 3.691666666666667 N


Сейчас мы рассмотрели вариант оплаты по заданиям, но не было учтено кое-что важное - вероятно, что когда задание стало зарезервированно за ассесором, он всё ещё может выполнять прошлое задание и получается, что текущая задача будет простаивать. 

Чтобы это учесть, мы будем смотреть только на объединение временных отрезков выполнения заданий, тк ассесор способен выполнять лишь одну задачу в один момент времени.

In [20]:
sliced_data = data[['login', 'assigned_ts', 'closed_ts', 'Microtasks']].sort_values(by=['login', 'assigned_ts', 'closed_ts'])

В данном случае нам важны лишь логин, начало и конец временного отрезка и количество микрозаданий. Сгруппируем данные по логину ассесора так, чтобы временные отрезки были закреплены за одним пользователем в виде списка. 

In [30]:
sd = sliced_data.groupby('login')['assigned_ts', 'closed_ts', 'Microtasks'].apply(lambda x: x.values.tolist()).reset_index()
sd.columns = ['login', 'start_end_microtasks']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [31]:
def union(start_end):
  x = []
  microtasks = 0
  for i in start_end:
    microtasks += i[2]
    x.append((i[0], False))
    x.append((i[1], True))
  x.sort()
  result = datetime(1970, 1, 1)
  c = 0
  for i in range(len(x)):
    if c and i:
      result += (x[i][0] - x[i-1][0])
    if x[i][1]:
      c+=1
    else:
      c-=1
  return (result - datetime(1970, 1, 1)).total_seconds() // microtasks

In [ ]:
time_for_mtasks_by_login = []
for index, row in sd.iterrows():
  time_for_mtasks_by_login.append((row['login'], union(row['start_end_microtasks'])))

In [50]:
time_for_mtasks_by_login_df = pd.DataFrame(time_for_mtasks_by_login, columns =['login', 'time'])
median_time_third = time_for_mtasks_by_login_df.time.median()
median_time_third

198.0

In [52]:
payment_third = get_payment(median_time_third)
print("Оплата за одно микрозадание(третий вариант) =", payment_third, 'N')

Оплата за одно микрозадание(третий вариант) = 6.6 N


Медиана окончательных данных времени выполнения микрозадания всех пользователей меня удивила, так как я ожидал, что она будет меньше, чем в первых двух случаях без объединения временных отрезков, по крайней мере не больше, вероятно, это из-за того, что в функции union() я беру среднее арифметическое время выполнения микрозадания, что поддается влияюнию выбросов. 

Напишу функцию union_median(), где буду считать по-другому. А именно если в union() я считал общее время в объединении этих отрезков и просто делил на количество микрозаданий на этих отрезках, получая при этом среднее арифметическое, то в union_median() я брал один объединенный отрезок, количество микрозаданий на получившемся отрезке, вычислял среднее на этом отрезке и добавлял в итоговый список. После этого в списке уже брал медиану по всем непересекающимся объединенным отрезкам.

In [59]:
def union_median(start_end_microtasks):
    ends_of_segment = []
    for i in start_end_microtasks:
        ends_of_segment.append((i[0], False, i[2]))
        ends_of_segment.append((i[1], True, i[2]))
    ends_of_segment.sort()
    sum_time_date = datetime(1970, 1, 1)
    last_sum_time_date = sum_time_date
    balance = 0
    count_mtasks = 0
    left_end_mtasks = ends_of_segment[0][2]
    times_for_mtask_median = []
    for i in range(len(ends_of_segment)):
        if balance and i:
            sum_time_date += (ends_of_segment[i][0] - ends_of_segment[i - 1][0])
            count_mtasks += ends_of_segment[i][2] + ends_of_segment[i - 1][2]
        if not balance and i:
            right_end_mtasks = ends_of_segment[i - 1][2]
            result_count_mtasks = (count_mtasks + left_end_mtasks + right_end_mtasks) // 4
            time_for_mtask = (sum_time_date - last_sum_time_date).total_seconds() // result_count_mtasks
            times_for_mtask_median.append(time_for_mtask)
            last_sum_time_date = sum_time_date
            left_end_mtasks = ends_of_segment[i][2]
            count_mtasks = 0
        if ends_of_segment[i][1]:
            balance += 1
        else:
            balance -= 1
    right_end_mtasks = ends_of_segment[i][2]
    result_count_mtasks = (count_mtasks + left_end_mtasks + right_end_mtasks) // 4
    time_for_mtask = (sum_time_date - last_sum_time_date).total_seconds() // result_count_mtasks
    times_for_mtask_median.append(time_for_mtask)
    return pd.Series(times_for_mtask_median).median()


In [61]:
time_for_mtasks_by_login = []
for index, row in sd.iterrows():
  time_for_mtasks_by_login.append((row['login'], union_median(row['start_end_microtasks'])))


In [56]:
time_for_mtasks_by_login_df = pd.DataFrame(time_for_mtasks_by_login, columns =['login', 'time'])
median_time_four = time_for_mtasks_by_login_df.time.median()
median_time_four

96.0

In [58]:
payment_four = get_payment(median_time_four)
print("Оплата за одно микрозадание(четвертый вариант) =", payment_four, 'N')

Оплата за одно микрозадание(четвертый вариант) = 3.2 N


Как итог наименее подвежренными выбросам варинтами являются второй и четвертый, в которых плата за одно микрозадание составляет 3.7N и 3.2N соответственно.